In [2]:
import fastf1
import numpy as np
import pandas as pd 
import os 
import matplotlib.pyplot as plt 
import sklearn 
import time 

In [8]:
df = pd.read_csv('data/f1_data_Japanese_Grand_Prix_2025.csv')
df.head()

,Driver,LapTime_FP1,Compound_FP1,SpeedST_FP1,SpeedFL_FP1,SpeedI1_FP1,SpeedI2_FP1,LongestStintCompound_FP1,LongestStintLaps_FP1,AvgLapTime_LongestStint_FP1,...,Q2_Pressure,Q3_TopSpeedST,Q3_AirTemp,Q3_TrackTemp,Q3_WindSpeed,Q3_WindDirection,Q3_Humidity,Q3_Pressure,TotalRaceTime,RacePosition
0,ALB,0 days 00:01:29.392000,SOFT,305.0,274.0,278.0,307.0,MEDIUM,7.0,0 days 00:01:34.412250,...,1013.3,294.0,14.4,28.7,3.6,149.0,60.0,1013.2,0 days 00:00:40.367000,9.0
1,ALO,0 days 00:01:29.222000,SOFT,302.0,271.0,275.0,304.0,HARD,7.0,0 days 00:01:31.104000,...,1013.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:57.333000,11.0
2,ANT,0 days 00:01:29.284000,SOFT,303.0,271.0,280.0,307.0,MEDIUM,13.0,0 days 00:01:34.429888888,...,1013.3,295.0,14.3,28.8,3.4,155.0,60.0,1013.3,0 days 00:00:18.671000,6.0
3,BEA,0 days 00:01:30.077000,SOFT,301.0,267.0,278.0,301.0,MEDIUM,9.0,0 days 00:01:32.337500,...,1013.3,294.0,14.3,28.8,3.4,155.0,60.0,1013.3,0 days 00:00:54.529000,10.0
4,BOR,0 days 00:01:30.147000,SOFT,300.0,268.0,277.0,305.0,HARD,13.0,0 days 00:01:34.285300,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:01:23.897000,19.0


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Columns: 103 entries, Driver to RacePosition
dtypes: float64(80), int64(3), object(20)
memory usage: 16.2+ KB


TotalRaceTime column works as follows: the driver that won the GP has the total race time written, for other drivers we have written the time delta to the race winner, also time not given if more than 1 lap behind leader 

1. rewrite total race time -> change it to seconds + actually write the whole time 
2. for all lap times -> rewrite them as seconds
3. rewrite compound as categorical variable -> soft 1, medium 2, hard 3
4. for spped just difference from slowest 

In [32]:
# for every lap time data 
df["LapTime_FP1"] = pd.to_timedelta(df["LapTime_FP1"]).dt.total_seconds()
df["LapTime_FP2"] = pd.to_timedelta(df["LapTime_FP2"]).dt.total_seconds()
df["LapTime_FP3"] = pd.to_timedelta(df["LapTime_FP3"]).dt.total_seconds()
df["AvgLapTime_LongestStint_FP1"] = pd.to_timedelta(df["AvgLapTime_LongestStint_FP1"]).dt.total_seconds()
df["AvgLapTime_LongestStint_FP2"] = pd.to_timedelta(df["AvgLapTime_LongestStint_FP2"]).dt.total_seconds()
df["AvgLapTime_LongestStint_FP3"] = pd.to_timedelta(df["AvgLapTime_LongestStint_FP3"]).dt.total_seconds()
df["Delta_FirstLastLap_FP1"] = pd.to_timedelta(df["Delta_FirstLastLap_FP1"]).dt.total_seconds()
df["Delta_FirstLastLap_FP2"] = pd.to_timedelta(df["Delta_FirstLastLap_FP2"]).dt.total_seconds() 
df["Delta_FirstLastLap_FP3"] = pd.to_timedelta(df["Delta_FirstLastLap_FP3"]).dt.total_seconds()
df["Q1_FastestLap"] = pd.to_timedelta(df["Q1_FastestLap"]).dt.total_seconds()
df["Q2_FastestLap"] = pd.to_timedelta(df["Q2_FastestLap"]).dt.total_seconds()
df["Q3_FastestLap"] = pd.to_timedelta(df["Q3_FastestLap"]).dt.total_seconds()


In [33]:
df["FastestPracticeTime"] = df[["LapTime_FP1", "LapTime_FP2", "LapTime_FP3"]].min(axis=1)

In [ ]:
# compounds to categorical variable 
compounds = {"SOFT": 0, "MEDIUM": 1, "HARD": 2, "INTERMEDIATE": 3, "WET": 4}


In [34]:
# total race time 
df = df.dropna(subset=["TotalRaceTime"])
df.head()

,Driver,LapTime_FP1,Compound_FP1,SpeedST_FP1,SpeedFL_FP1,SpeedI1_FP1,SpeedI2_FP1,LongestStintCompound_FP1,LongestStintLaps_FP1,AvgLapTime_LongestStint_FP1,...,Q3_TopSpeedST,Q3_AirTemp,Q3_TrackTemp,Q3_WindSpeed,Q3_WindDirection,Q3_Humidity,Q3_Pressure,TotalRaceTime,RacePosition,FastestPracticeTime
0,ALB,77.713,MEDIUM,322.0,306.0,286.0,321.0,MEDIUM,7,98.290667,...,331.0,31.2,40.5,0.4,105.0,34.0,1009.0,0 days 00:00:12.773000,5.0,76.258
2,ANT,78.390,MEDIUM,318.0,306.0,284.0,318.0,MEDIUM,10,83.281571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:10.135000,4.0,76.206
3,BEA,79.312,MEDIUM,305.0,299.0,277.0,312.0,MEDIUM,10,99.552333,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 days 00:00:40.351000,14.0,79.312
6,GAS,78.505,MEDIUM,320.0,304.0,279.0,317.0,MEDIUM,10,83.658571,...,326.0,31.2,40.5,0.4,105.0,34.0,1009.0,0 days 00:00:26.502000,11.0,76.719
8,HAM,78.071,MEDIUM,321.0,306.0,284.0,317.0,MEDIUM,7,99.719167,...,328.0,31.2,40.5,0.4,105.0,34.0,1009.0,0 days 00:00:22.473000,10.0,76.378


In [ ]:
# rewrite speed data as how much faster compared to slowest 
min_speed = df["SpeedST_FP1"].min()
df["SpeedST_FP1"] = df["SpeedST_FP1"] - min_speed
df["SpeedST_FP1"].head()



0    17.0
2    13.0
3     0.0
6    15.0
8    16.0
Name: SpeedST_FP1, dtype: float64

In [36]:
# fastest lap during qualifying 
df["FastestLapQualy"] = df[["Q1_FastestLap", "Q2_FastestLap", "Q3_FastestLap"]].min(axis=1)

In [ ]:
df["QualifyingPosition"] # not really important 

0      6.0
2     16.0
3     20.0
6      9.0
8      8.0
9     17.0
11     7.0
12     1.0
13    19.0
14     2.0
15     4.0
17    13.0
18     5.0
19     3.0
Name: QualifyingPosition, dtype: float64

DATA ANALYSIS 
- correlation matrix 